# 1. Боты общего назначения

# 2. Goal-oriented боты

Доступные датасеты:

* [Dialog State Tracking Challenge](https://www.microsoft.com/en-us/research/event/dialog-state-tracking-challenge/)
* [ATIS](https://github.com/mesnilgr/is13#atis-data)

##### Общая схема

<img src="img/scheme.png" width="45%">

## Подходы

* Callflow
* Formfilling
* Reinforcement Learning
* Supervised Learning
* Supervised Learning + Reinforcement Learning

Разберем их подробнее.

### Callflow

Callflow = конечный автомат. Работает по схеме: _"Спросить первое", "спросить второе", "конец"_.

__Плюсы__:

+ Нагляден.
+ Может делать непрограммист.

__Минусы__:

- Не учитывает модели поведения: _"Здравствуйте, я хочу такси"_ или _"На Льва Толстого, 16"_ - способы начать один и тот же реквест.
- Не учитывает накопленный опыт.

## Formfilling

| Слот | Содержимое |
|:----|:---------|
| Откуда | \_\_\_\_\_\_\_\_ |
| Куда | \_\_\_\_\_\_\_\_ |
| Когда | \_\_\_\_\_\_\_\_ |

etc.

##### Formfilling NLU
  
* __Классификатор интентов__
      
      "Мне нужно заказать такси" -> `Заказ_такси`
      "Какая погода на сегодня?"  -> `Прогноз_погоды`
      
    * Алгоритмы: `LogReg`, `NN`, ...  
  
  
* __Теггер слотов__

      "Такси давай на улицу Толстого, 16"
                        ^^^^^^^^^^^^

    * Алгоритмы: `CRF`, `RNN`, ...

##### Natural Language Generation

* __Подтвердить:__
      Откуда="улица Толстого"

* __Результат__
      "Вы заказали такси на улицу Толстого. Все верно?"

    * Алгоритмы: `Контекстно-свободные грамматики`, `NN`

##### Dialogue Manager

Связывает все вместе:

1. Создает форму
2. Инициирует действия
3. etc.

###### Вложенные формы

Когда пользователь начинает, например, бронировать отель, а потом переспрашивает _"а какая там погода?"_

* Формируется __стек форм__: то, о чем сейчас говорим, на верхушке стека.
* __Переиспользование данных__: в примере выше то, где прогноз погоды должен быть взят, может быть выделено из предыдущей формы, что осталась ниже в стеке.

__Плюсы__

* Пользователь может говорить, как ему удобно
* Вся информация попадает в форму

__Минусы__

* DM логика из скриптов 
* Не учитывает ошибки ASR
* Действия на основе интуиции дизайна диалога 

_(У них как-то делался в языке AML)_

## Reinforcement learning

Цель:

* За минимальное количество шагов
* Учитывая ошибки ASR

<img src="img/RL-based dialogue.png" width="50%">

RL в диалогах = 2 задачи:

1. State Tracking / Belief Monitoring
  - цель пользователя -> скрытое состояние (belief state)
  - Байесовский вывод для трекинга belief state

2. Action Selection / Policy Optimization
  - belief state -> action (policy)
  - RL оптимизирует политику

Belief state = вектор из вероятностей находиться в таком-то состоянии для каждого из слота.

<img src="img/belief state.png" width="40%">

Дальше RL, который уточняет belief, если надо, и таким образом минимум вопросов.
Что-то вроде Q-table.

<img src="img/two state example.png" width="50%">

<img src="img/cumulative reward.png" width="40%">

__Плюсы__

* Учитывает модели пользовательского поведения
* Действия на основе минимального количества шагов
* Учитывает ошибки ASR
* Может обучаться в production на миллионах пользователей

__Минусы__

* Необходим симулятор пользователя
* Долго учится

## Supervised Learning

* Data driven подход
* __Вход__: размеченные диалоги (NLU, NLG, Actions)
* __Выход__: Actions (NLG, API calls)
* __Задача__: реконструирование диалога

* __Алгоритмы__: RNN всякие

Может быть совмещено с RL. Но в продакшне ни у кого такого нет.

## Заключение

* Callflow не гибко.
* Formfilling'у нужно больше масштабируемости.
* RL+SL каеф, но сложна.